In [134]:
import csv
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
# エンコーディング自動検出用
from charset_normalizer import from_path

In [135]:
# HTMLファイルが格納されているディレクトリのパス
directory = "www.geocities.co.jp/Playtown-Dice/6159"

In [136]:
def is_mojibake(text):
    """
    簡易的な文字化け検出関数。非表示文字や無効な文字がある場合にTrueを返す。
    """
    # 文字化けによく見られる文字パターンを除外（ここでは基本的な制御文字を例に）
    mojibake_patterns = [
        r'[�]'  # 置換文字
    ]
    
    for pattern in mojibake_patterns:
        if re.search(pattern, text):
            return True
    return False

In [137]:
def extract_all_entries_from_html(file_path):
    """
    HTML内のすべての trペア から {title, issue_info, body_text} を抽出する
    """
    try:
        result = from_path(file_path).best()
        content = str(result)
        soup = BeautifulSoup(content, 'html.parser')

        rows = soup.find_all("tr")
        entries = []

        # ペアで処理（title/infoが1行目, 本文が2行目）
        for i in range(0, len(rows) - 1, 2):
            tr_title = rows[i]
            tr_body = rows[i + 1]

            # タイトル
            title_tag = tr_title.find("font", attrs={"size": "+2"})
            title = title_tag.get_text(strip=True) if title_tag else None

            # 出典
            issue_tag = tr_title.find("font", attrs={"size": "-1"})
            issue_info = issue_tag.get_text(strip=True) if issue_tag else None

            # 本文（1つまたは複数）
            # span.line タグが壊れてる可能性に備え、tr_body 全体を文字列として処理
            raw_html = str(tr_body)

            # <span class="line">〜(次の <span> or </tr> or <td> まで)
            raw_spans = re.findall(r'<span class="line">(.*?)(?=<span class="line">|<td bgcolor="#80ffff">|</tr>|</td>)', raw_html, re.DOTALL)
            # HTMLタグを除去して本文だけにする
            body_texts = [BeautifulSoup(fragment, "html.parser").get_text(separator=" ", strip=True) for fragment in raw_spans]

            body_text = body_texts[0]

            full_text = f"{title or ''}\n{issue_info or ''}\n{body_text or ''}"
            if is_mojibake(full_text):
                print(f"Skipping entry due to mojibake at index {i}")
                continue

            entries.append({
                "title": title,
                "issue_info": issue_info,
                "body_text": body_text
            })

        return entries

    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return []

In [138]:
def extract_d_xx_recursive_to_df(directory, max_pages=1000):
    """
    指定されたディレクトリを再帰的に検索し、HTMLファイルからテキストを抽出し、
    ディレクトリ構造をキーとしてDataFrameに格納する関数
    """
    data = []
    page_count = 0
    
    for root, _dirs, files in os.walk(directory):
        for file in files:
            if re.match(r"^d-\d+\.html?$", file):
                file_path = os.path.join(root, file)
                stories = extract_all_entries_from_html(file_path)
                if len(stories) == 0: continue
                # ディレクトリ構造と抽出したテキストをDataFrameに格納するためリストに追加
                for index, story in enumerate(stories, 1):
                    data.append({
                        "directory": root,
                        # "dirs": _dirs,
                        "file": file,
                        "story_index": index,
                        "title": story["title"],
                        "issue_info": story["issue_info"],
                        "body_text": story["body_text"]
                    })
                    
                page_count += 1
                if page_count >= max_pages:
                    print(f"Processed {max_pages} pages. Stopping...")
                    return pd.DataFrame(data)  # DataFrameに変換して返す
    
    return pd.DataFrame(data)

In [139]:
# 実行
df = extract_d_xx_recursive_to_df(directory)

In [140]:
df.sort_values(by=["file", "story_index"], inplace=True, ignore_index=True)
# データフレームを表示
print(df.head())

                                directory      file  story_index        title  \
0  www.geocities.co.jp/Playtown-Dice/6159  d-1.html            1  未来の国からはるばると   
1  www.geocities.co.jp/Playtown-Dice/6159  d-1.html            2    ドラえもんの大予言   
2  www.geocities.co.jp/Playtown-Dice/6159  d-1.html            3      変身ビスケット   
3  www.geocities.co.jp/Playtown-Dice/6159  d-1.html            4      秘スパイ大作戦   
4  www.geocities.co.jp/Playtown-Dice/6159  d-1.html            5         コベアベ   

                 issue_info                                          body_text  
0  １５頁　小四７０年１月号（ドラえもんあらわれる）  お正月。のどかに自分の部屋でモチを食べる少年、野比のび太。そこへ突然どこからともなく、「三十...  
1              １４頁　小四７０年２月号  いつものように学校から帰ってきて出かけようとするのび太を、ドラえもんが２２世紀のマジックハン...  
2      ９頁　小六７４年４月号（動物ビスケット）  ママからお客にお菓子を買ってくるように頼まれたのび太。ドラえもんに行かせようとするがドラえも...  
3              １４頁　小四７０年５月号  誤って先生のカビンを割ってしまったのび太。のび太は先生に謝りに行こうとするが、それを見ていた...  
4               ９頁　小四７２年４月号  自分の部屋を片付けないでしずかのところへ出かけようとするのび太を叱るママ。その時ドラえもんが...  


In [141]:
df.to_csv("doraemon_stories.csv", index=False, escapechar='\\', quoting=csv.QUOTE_ALL)